# Data extraction

This notebook describes the data extraction steps that were undertaken to get our final dataset. Approaches that were tried but ended up _not_ being used are also included.

## Setup

In [2]:
# Built-in
import json
import bz2
import os
import csv
import re

# Third parties
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
# Initialization needed for some modules

# tqdm for pandas
tqdm.pandas()

In [4]:
# Configuration
DATA_PATH = "data"
PKL_PATH = os.path.join(DATA_PATH, "pkl")
CSV_PATH = os.path.join(DATA_PATH, "csv")
RESOURCES_PATH = os.path.join(DATA_PATH, "resources")

In [5]:
# Utils functions

def to_csv(file_name: str, pol_lst: list) -> None:
    """
    Write list to csv.
    """

    csv_path = os.path.join(RESOURCES_PATH, file_name)

    with open(csv_path, "w", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=" ")
        writer.writerow(["Name", "Party"])

        for member in pol_lst:
            writer.writerow([el for el in member])
            
def get_pkl_year(year: int) -> list:
    """
    Returns a list of the pkl files present in `PKL_PATH/{year}`.
    """

    dirs = os.listdir(os.path.join(PKL_PATH, str(year)))

    return [os.path.join(str(year), dir) for dir in dirs]

def sanitize_name(name: str) -> str:
    """
    Strip and clean name.
    "Senator Cruz, Ted" -> "Ted Cruz"
    """

    for element in ("Representative", "Senator"):
        name = name.strip(element)

    name = " ".join(name.split(",")[::-1])
    name = name.strip()
    
    return name

## 1. Get list of US politicians

First of all, we need to have a list of current US politicians to be able to extract their quotes. After some research, we found two possible candidates:
- the official [US congress website](https://www.congress.gov/members?q={%22congress%22:[%22110%22,%22111%22,%22112%22,%22113%22,%22114%22,%22115%22,%22116%22,117]})
- a list of US politicians extracted from Twitter for a [research](https://github.com/casmlab/politicians-tweets). The politicians list is available on [Github](https://raw.githubusercontent.com/casmlab/politicians-tweets/main/metadata/usa/current.json). 

### US congress website

Since it is an official source, it should be reliable. The caveat is that no official API exists, so the content needs to be scraped. That is what is done in this section.

In [6]:
URL = 'https://www.congress.gov/members?q={"congress":["110","111","112","113","114","115","116",117]}&pageSize=250'
congress_members = []

# Download each congress page
with requests.Session() as s:
    for page_number in tqdm(range(1, 6)):
        r  = s.get(URL, params={"page": page_number})
        soup = BeautifulSoup(r.text, "html.parser")

        members = soup.find_all("li", class_="compact")

        for member in members:
            # Scrape the information
            items = member.find_all("span", class_="result-item")
            name = sanitize_name(member.span.a.text)
            
            for item in items:
                if item.strong.text == "Party:":
                    affiliation = item.span.text

            congress_members.append((name, affiliation))

100%|██████████| 5/5 [00:09<00:00,  1.88s/it]


In [7]:
# Sanity check
print(f"Number of retrieved congress members {len(congress_members)}")

Number of retrieved congress members 1160


In [8]:
# Write to file
to_csv("politicians_congress.csv", congress_members)

So we have 1160 congress members. This is satisfactory to start with. One caveat is that we only have the politician's name and party affiliation. We do not easily have more information (age, state, gender, ...) without matching the data with some other dataset (wikidata for example). This would mean additional work.

### Github list

This research [1] "collect tweets posted by politicians in the U.S. and India and save the JSON provided by the Twitter API. Lists of politicians are generated by NivaDuck, software developed at Microsoft Research - India for automatically identifying accounts that belong to politicians". 

In this section, we explore the given list to see if we could use it.

In [9]:
# Load the json file
file_name = "politicians_github.json"
file_path = os.path.join(RESOURCES_PATH, file_name)

with open(file_path, "r") as f:
    json = json.load(f)
    
# Print the keys to see what we have
print(f"Json columns {json.keys()}")

Json columns dict_keys(['id', 'id_str', 'screen_name', 'confirmed_account_type', 'state', 'twitter_name', 'real_name', 'bioguide', 'office_holder', 'party', 'district', 'level', 'woman', 'birthday', 'last_updated'])


As written in their research, they selected all "politicians" classified as such by the NivaDuck software. This means that some entries are not professional politicians (ie. congress members), but just influential people. We decide to only keep the official politicians.

In [10]:
# Only keep politicians with political affiliation
politicians = []

for i in range(1, len(json["id"])):
    i = str(i)  # index is a string in json
    affiliation = json["party"][i]
    screen_name = json["screen_name"][i]
    elected = json["office_holder"][i] is not None

    if affiliation is not None and affiliation in ("Republican", "Democratic"):
        politicians.append((json["real_name"][i], affiliation, elected))
    elif screen_name == "realdonaldtrump":
        politicians.append(("Donald Trump", "Republican", True))
    elif screen_name == "barackobama":
        politicians.append(("Barack Obama", "Democratic", True))
        
# Count how many politicians are "elected" (-> congress members)
elected_count = sum(pol[-1] for pol in politicians)
print(f"{elected_count} in the Github dataset")

1107 in the Github dataset


In [11]:
# Sanity check
print(f"{len(politicians)=}") 
politicians[:5]

len(politicians)=1107


[('Mark Green', 'Republican', True),
 ('Pete Stauber', 'Republican', True),
 ('Derek Kilmer', 'Democratic', True),
 ('Andy Harris', 'Republican', True),
 ('Donald Payne', 'Democratic', True)]

In this dataset, 1107 congress members (and Trump) are present. This is less than the 1158 from the official congress website.

In [12]:
# Write to file
to_csv("politicians_github.csv", politicians)

[1] Panda, A., Gonawela, A., Acharyya, S., Mishra, D., Mohapatra, M., Chandrasekaran, R., & Pal, J. (2020). NivaDuck - A Scalable Pipeline to Build a Database of Political Twitter Handles for India and the United States. International Conference on Social Media and Society, 200–209. https://doi.org/10.1145/3400806.3400830

### Final chosen politicians list

Since we have more politicians in the official congress' list, we should keep that one. As already explained, one caveat is that we don't have much information apart from the politician's name. 

However, we are lucky. An official list of congress members with plenty information does exist! In fact, it was even mentioned on the project's page, what a shame that we did not see it sooner (_sigh_). 

This resource is the official biography list of the congress: [congress list](https://bioguide.congress.gov/search?index=%22bioguideprofiles%22&size=12&matches=%5B%5D&filters=%7B%22jobPositions.congressAffiliation.partyAffiliation.party.name%22:%5B%22Democrat%22,%22Republican%22%5D,%22jobPositions.congressAffiliation.congress.name%22:%5B%22The%20110th%20United%20States%20Congress%22,%22The%20111th%20United%20States%20Congress%22,%22The%20112th%20United%20States%20Congress%22,%22The%20113th%20United%20States%20Congress%22,%22The%20114th%20United%20States%20Congress%22,%22The%20115th%20United%20States%20Congress%22,%22The%20116th%20United%20States%20Congress%22,%22The%20117th%20United%20States%20Congress%22%5D%7D&sort=%5B%7B%22_score%22:true%7D,%7B%22field%22:%22familyName%22,%22order%22:%22asc%22%7D,%7B%22field%22:%22middleName%22,%22order%22:%22asc%22%7D,%7B%22field%22:%22givenName%22,%22order%22:%22asc%22%7D%5D). We select the congress members from 2007 up to today. 

The data can be directly exported as `json`. Also we have access to the "congress bio ID" of each congress member, which is also present in the `speaker_attributes.parquet` file (`field US_congress_bio_ID`). Even if we don't use that directly, we can use that information later on.

In [13]:
# Load the list
politicians_filepath = os.path.join(RESOURCES_PATH, "congress_biolist.json")
politicians_df = pd.read_json(politicians_filepath)

One issue that was detected (which also happens for the first official congress list) is that Donald Trump is not in the dataset (because he was President, not senator or representative and thus, not a congress member). As he is expected to be the `speaker` of many quotes, we will need to manually add him.

In [14]:
# Manually add Donald Trump
# Not so elegant trick to capture variations of the name
# Should refactore to another solution (alias field of speaker_attributes) later

donald_json1 = {
    "id": np.nan,
    "givenName": "Donald",
    "familyName": "Trump",
    "unaccentedGivenName": "Donald",
    "unaccentedFamilyName": "Trump",
    "birthYear": 1946,
    "deathYear": np.nan,
    "congresses": [
        {
            "position": "President",
            "congressNumber": np.nan,
            "stateName": np.nan,
            "parties": ["Republican"],
        }
    ],
    "middleName": "John",
    "unaccentedMiddleName": "John",
    "nickName": np.nan,
    "honorificPrefix": np.nan,
    "honorificSuffix": np.nan,
}

donald_json2 = donald_json1.copy()
donald_json2["givenName"] = "President"
donald_json2["unaccentedGivenName"] = "President"

donald_json3 = donald_json1.copy()
donald_json3["givenName"] = "President Donald"
donald_json3["unaccentedGivenName"] = "President Donald"

politicians_df = politicians_df.append(
    pd.DataFrame([donald_json1, donald_json2, donald_json3]), ignore_index=True
)

In [15]:
# Check that Trump was indeed appended to the df
politicians_df.tail(4)

,id,givenName,familyName,unaccentedGivenName,unaccentedFamilyName,birthYear,deathYear,congresses,middleName,unaccentedMiddleName,nickName,honorificPrefix,honorificSuffix
1154,Z000018,Ryan,Zinke,Ryan,Zinke,1961,NaN,"[{'position': 'Representative', 'congressNumbe...",NaN,NaN,NaN,NaN,NaN
1155,NaN,Donald,Trump,Donald,Trump,1946,NaN,"[{'position': 'President', 'congressNumber': n...",John,John,NaN,NaN,NaN
1156,NaN,President,Trump,President,Trump,1946,NaN,"[{'position': 'President', 'congressNumber': n...",John,John,NaN,NaN,NaN
1157,NaN,President Donald,Trump,President Donald,Trump,1946,NaN,"[{'position': 'President', 'congressNumber': n...",John,John,NaN,NaN,NaN


In [16]:
# Export new df to json
politicians_df.to_json(os.path.join(RESOURCES_PATH, "new_congress_biolist.json"))

This dirty trick allows us to capture quotes from Donald Trump and its most appearing variations ("Donald Trump", "President Trump", "President Donald Trump"). The complex structure of the json is there to match the initial congress biography list.

As mentioned in the comment, a more elegant solution can (and will) be implemented using the `speaker_attributes` `alias` field. 

## 2. Extract quotes from politicians

Since we now have our list of politicians, we need to extract their quotes from the Quotebank dataset (quotation centric).
To do that, and to split the load between team members, we decided to first find a way to load the quotebank dataset of a given year into pandas, which is not possible to do in one chunk because of memory limitations.

In [18]:
# Functions needed for the task

def load_df(
    file_name: str, mode: str = "pandas", save: bool = True, chunksize: int = 500_000
) -> pd.DataFrame:
    """
    Load a dataset in DataFrame from a .json.bz2 archive.

    file_name: str
        Name of .json.bz2 archive to load from `DATA_PATH`.

    mode: str = "pandas" | "bz2"
        Either use pandas read_json function or homemade bz2 function. This is usually faster (but makes my computer crash for some reason).
        Mode "bz2" should be used if you are sure that the dataframe can fit into memory.

    save: bool
        Save the dataframe as a pickle file in `PKL_PATH`.
    """

    file_path = os.path.join(DATA_PATH, file_name)

    # Check if file exists
    if not os.path.exists(file_path):
        print("File not found")
        return
    
    # Extract yer from file_name
    year_re = r"20\d\d"
    year_file = re.search(year_re, file_name).group(0)

    if mode == "bz2":  # Only use if can fit in memory!
        if save:
            # Be sure to inform the user that we are not saving
            # even though flag is set
            print("Save option currently not supported for \"bz2\" mode.")
        
        # Subset of keys to load
        keys = ["quoteID", "quotation", "speaker", "date", "numOccurrences", "phase"]  

        with bz2.open(file_path, "rb") as quote_file:
            df = pd.DataFrame(
                [
                    dict(zip(keys, map(json.loads(instance).get, keys)))
                    for instance in tqdm(quote_file)
                ]
            )
        
        return df
            
    else:  # pandas load
        if not save:  # force the need to save 
            print("Please enable save option.")
            return
        
        # Load in chunks and save to pickle
        with pd.read_json(file_path, lines=True, chunksize=chunksize) as df_reader:
            for i, chunk in enumerate(df_reader):
                file_name = file_name.strip(".json.bz2")
                pkl_path = os.path.join(PKL_PATH, year_file, f"{file_name}-{i:03d}.pkl")
                chunk.to_pickle(pkl_path)
    
        # If we use pandas, we only return the last chunk (for debugging)
        return chunk

def extract_subset(orig_df: pd.DataFrame, multiproc=False) -> pd.DataFrame:
    """
    This function extracts the quotes of speakers that are in the congress list.
    It returns the number of extracted quotes and the extracted dataframe.
    
    Multiprocessing is supported. Set to False if any issue is encountered.
    """

    if multiproc:
        # Load module and initialize only if we need it
        from pandarallel import pandarallel
        pandarallel.initialize(progress_bar=True)
        
        orig_df["subset"] = orig_df["speaker"].parallel_apply(
            lambda x: pd.Series(x.lower()).str.contains("|".join(congress_members))
        )
    else:
        orig_df["subset"] = orig_df["speaker"].progress_apply(
            lambda x: pd.Series(x.lower()).str.contains("|".join(congress_members))
        )

    return orig_df["subset"].sum(), orig_df[orig_df["subset"] == True]

In [ ]:
##
# Do not run this cell, it takes (many) hours.
# It was run once with this exact code to generate the working dataframes.
##

# Load each .json.bz2 archive, load it in chunks, convert to pd.DataFrame and save to pickle

archives = [f"quotes-20{i:02d}.json.bz2" for i in range(15, 21)]  
# The following code is disabled such that this heavy code does not run if someone runs the cell on accident. 
# Change False to True to enable the code
if False:
    for i, archive in enumerate(archives, start=1):
        print(f"{i}/{len(archives)} {archive}:")
        try:
            load_df(archive)
        except FileNotFoundError:
            print(f"{archive} not found, going to next file")
            continue

Now that we have access to chunks of the total dataset of each year, we can simply loop through each chunk to extract the relevant quotes. 

In [20]:
# Create a new column from politicians to match the speaker field from quotebank
politicians_df["fullName"] = politicians_df["givenName"] + " " + politicians_df["familyName"]
politicians_df["fullName"] = politicians_df["fullName"].str.lower()

congress_members = politicians_df["fullName"].tolist() 

In [ ]:
##
# Also computation-heavy cell. 
##

# The datasets were already loaded from the json.bz2 format and converted to .pkl in in chunks in `data/pkl/{year}`

# Loop through each year of interest
for year_i in range(2015, 2021):
    print(year_i)
    
    # Get all the chunks for the given year
    files = get_pkl_year(year_i)

    # Extract the quotes of interest of each chunk
    all_extracted = []
    for file in files:
        df = pd.read_pickle(os.path.join(PKL_PATH, file))
        _, subset_df = extract_subset(df)
        all_extracted.append(subset_df)

    # Merge them into a new df
    df_extracted = pd.concat(all_extracted)
    
    # Print sanity check
    print(f"{len(df_extracted)=}")

    # Save the df as pkl
    pkl_name = f"extracted-quotes-{year_i}.pkl"
    df_extracted.to_pickle(os.path.join(PKL_PATH, pkl_name))

Nice! We manage to handle the huge size of the data and we are getting to some manageable size.

## 3. Extract mentions

The last thing that we need to do is to extract the quotes of politicians that are mentioning a policitian from the other party, eg. a republican mentioning a democrat, or inversely. 

### Work on politicians_df

First, the data that we need from the politicians list is in a "raw" format. We want to have it in a "clean" way.

In [22]:
# Extract from congresses column relevant infor (position, state, parties)
def extract_congress_information(row):
    information = pd.json_normalize(row["congresses"]).sort_values("congressNumber").tail(1).loc[:, ["position","stateName","parties"]]

    row["position"] = information.loc[:, "position"].values[0]
    row["stateName"] = information.loc[:, "stateName"].values[0]
    row["parties"] = information.loc[:, "parties"].values[0]
    return row

politicians_df = politicians_df.progress_apply(extract_congress_information, axis=1)

100%|██████████| 1158/1158 [00:03<00:00, 361.10it/s]


In [23]:
# The value in column parties is a list, we want to select the last past party from the list
def getLastValue(aList):
    return aList[-1]

politicians_df["parties"] = politicians_df["parties"].apply(getLastValue)

In [25]:
# Create new column speaker (full name of the politician)
politicians_df['speaker'] = politicians_df['givenName'] + " " + politicians_df['familyName']

# Have the speaker's full names in the same size 
politicians_df["speaker"] = politicians_df["speaker"].str.lower()

In [26]:
# Check for full name duplicates
politicians_df['speaker'].value_counts()

timothy johnson           2
donald payne              2
duncan hunter             2
patrick murphy            2
gregory murphy            1
                         ..
ruben gallego             1
pete gallego              1
elton gallegly            1
michael gallagher         1
president donald trump    1
Name: speaker, Length: 1154, dtype: int64

The above cell refers to the remark in the README file. For now, we decide to remove those "duplicates".

In [27]:
# We drop duplicates by speaker's full name
politicians_df = politicians_df.drop_duplicates(subset=['speaker'], keep=False)

### Merge and select subset

Then, we want to merge the quotes dataset of a given year with the politicians dataset and then directly select quotes that are mentioning politicians from the other camp. Below is the whole pipeline to do that.

That code is proposed for a single year. We ran it for every year of interest to extract the relevant quotes.

In [28]:
YEAR = "2015"

# Load the main dataset
df = pd.read_pickle(os.path.join(PKL_PATH, f"extracted-quotes-{YEAR}.pkl"))

In [29]:
# Set speaker's name to lowercase
df["speaker"] = df["speaker"].str.lower()

# Merge quotes to speaker's info
data = pd.merge(df, politicians_df, on='speaker', how='outer')

# Subsets by parties 
subset_democrats = data[data['parties'] == "Democrat"]
subset_republicans = data[data['parties'] == "Republican"]

# Sanity checks
print(f"{len(subset_democrats)=}")
print(f"{len(subset_republicans)=}")

len(subset_democrats)=166494
len(subset_republicans)=253528


Then, out of those subsets, we want to extract only the quotes that are mentioning a politician from the other camp. For now, we are limiting ourselves to a "naive" approach, where we only detect when the full name (given name + family name in lowercase) is mentioned. Also note that we added the detection of "republican(s)" or "democrat(s)".

In [33]:
# Get the list of Republicans and Democrats
names_democrats = politicians_df[politicians_df['parties'] == 'Democrat']["speaker"].tolist()
names_republicans = politicians_df[politicians_df['parties'] == 'Republican']["speaker"].tolist()

# Ensure that all quotes are in lowercase
subset_democrats.loc[:, "quotation"] = subset_democrats["quotation"].str.lower()
subset_republicans.loc[:, "quotation"] = subset_republicans["quotation"].str.lower()

In [34]:
# Drop rows with those who don't have any quotes (no speaker detected)
subset_democrats = subset_democrats.dropna(subset=['quotation'])
subset_republicans = subset_republicans.dropna(subset=['quotation'])

In [35]:
# Create full lists that we want to match against
list_rep = names_republicans + ['republicans?']
pattern_list_rep = '|'.join(list_rep)

list_dem = names_democrats + ['democrats?']
pattern_list_dem = '|'.join(list_dem)

In [36]:
# The actual extraction
# Subset of quotes said by democrats about republicans
# This cell takes ~1min to complete
demo_quotes_abt_rep = subset_democrats[subset_democrats['quotation'].str.contains(pattern_list_rep)]

In [37]:
# Subset of quotes said by democrats about democrats
# This cell takes ~1min to complete
rep_quotes_abt_demo = subset_republicans[subset_republicans['quotation'].str.contains(pattern_list_dem)]

We simply then save the extracted quotes for each year to csv. The csv format was preferred for this task because we encountered some issues when using pickle files on different machines.

In [38]:
# Save to CSV
df_to_save = pd.concat([demo_quotes_abt_rep, rep_quotes_abt_demo])
file_path = os.path.join(CSV_PATH, f"{YEAR}_mentions.csv")
df_to_save.to_csv(file_path)